## Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas 
import os
import sys
collection_name = sys.argv[1]
db_name = sys.argv[2]

## Read Config File

In [ ]:
import configparser 

config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
output_folder = config['DEFAULT']['Output-Folder']

## Connect MongoDB

In [ ]:
from pymongo import MongoClient
client = MongoClient(ip, int(port))
db_twitter = client[db_name]

In [ ]:
# get number of records
print("{} : {}".format(collection_name,db_twitter[collection_name].find({}).count()))

## Pipeline

In [ ]:
pipeline = [
    {"$match": { "user.location": {"$exists":True,"$ne":''}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": {
            "user_location": "$user.location"
        },
        "count": { "$sum": 1 },
        }
    }
]

## Get User Location

In [ ]:
# aggregate user_location data
data_list = list(db_twitter[collection_name].aggregate(pipeline,allowDiskUse=True))
print("Number of Unique User Location: "+ str(len(data_list)))

In [ ]:
# get list of user location
df= pandas.DataFrame(data_list)
user_location = df["_id"].tolist()
df_user_location= pandas.DataFrame(user_location)

## Supporting Functions

In [ ]:
# create foler if not exist
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

## Write into CSV

In [ ]:
create_folder(output_folder)
df_user_location.to_csv(output_folder + collection_name+".csv", sep=',',index = False, encoding='UTF-8')